# Moiré Cavity Exploration Pipeline (Square Lattice)

This notebook refactors the original proof-of-concept to use the provided square-lattice monolayer candidate (r = 0.43, ε = 4.02) and runs a cavity search via the envelope approximation.

What’s included:
- Use a square-lattice monolayer band edge, at the provided k0 and band index (read from the prior search results; no extremum search here)
- Robust stacking registry detection with visual confirmation (AA, ABx = (½,0), ABy = (0,½))
- Envelope-domain size option: 1, 2, or 3 moiré unit cells (default 1), plus a plot of the small moiré unit cell
- Pre-check run at a hard-coded test angle of 1.1° with full visualizations
- SciPy-based twist-angle optimization to maximize a cavity score (strongest bound state)

Effective equation used:

$$\Big[-\tfrac{\hbar^2}{2m^*}\nabla^2 + V(\mathbf r)\Big]F(\mathbf r) = (\omega - \omega_0)F(\mathbf r),$$

where $\omega_0$ and $m^*$ come from the monolayer at the supplied $(n, \mathbf k_0)$, and $V(\mathbf r)$ is built from registry-dependent band-edge shifts (AA/ABx/ABy).

Notes:
- If Meep is unavailable, the notebook will use placeholder values for $\omega_0$ and registry shifts to demonstrate the workflow.
- You can change the envelope domain via `ENVELOPE_DOMAIN_CELLS = 1|2|3` near the top of the notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection
from matplotlib.patches import Circle, Polygon
import moire_lattice_py as ml
import math
from typing import List, Tuple, Dict, Optional
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import eigsh
from scipy.optimize import minimize_scalar
import warnings

# Import meep for band calculations
try:
    import meep as mp
    from meep import mpb
    MEEP_AVAILABLE = True
    print("✓ Meep is available for band structure calculations")
except ImportError:
    MEEP_AVAILABLE = False
    print("✗ Meep not available - band structure calculations will be limited")
    warnings.warn("Meep not available. Install meep to run full pipeline.")

# Set up plotting parameters
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

print("🚀 Moire Cavity Exploration Pipeline Initialized (Square lattice)")
print("=" * 50)

In [ ]:
# Parameters and configuration for square lattice photonic crystal (candidate provided)
import numpy as np

# Base lattice
LATTICE_TYPE = "square"  # use square lattice
LATTICE_CONSTANT = 1.0

# Candidate from prior search (do not re-search here)
HOLE_RADIUS = 0.43          # radius in units of a
DIELECTRIC_CONST = 4.02     # background epsilon
POLARIZATION = "tm"         # target polarization for MPB

# Provided band-edge information (from pc_band_extrema_results)
# Square lattice: M-point corresponds to fractional k = (0.5, 0.5)
TARGET_K_FRAC = (0.5, 0.5)
TARGET_BAND_INDEX = 3       # 0-indexed band index at the extremum

# Test/pre-check twist angle (deg) before optimization
TEST_TWIST_ANGLE_DEG = 1.1

# Envelope domain options: number of moiré unit cells per side (1, 2, or 3)
ENVELOPE_DOMAIN_CELLS = 1   # default

# Discretization
GRID_SIZE = 64              # grid points per side in envelope solver domain
RESOLUTION = 32             # MPB pixels/a for any needed band calcs
NUM_BANDS = 8               # bands to compute when needed (for m* or registry deltas)

# SciPy optimization toggle
DO_TWIST_OPTIMIZATION = True

print("Physical Parameters (Square PC candidate):")
print(f"  Lattice constant a: {LATTICE_CONSTANT}")
print(f"  Hole radius r/a:    {HOLE_RADIUS}")
print(f"  Epsilon (bg):       {DIELECTRIC_CONST}")
print(f"  Target k (frac):    {TARGET_K_FRAC}")
print(f"  Target band index:  {TARGET_BAND_INDEX}")
print(f"  Test twist angle:   {TEST_TWIST_ANGLE_DEG}°")
print(f"  Envelope domain:    {ENVELOPE_DOMAIN_CELLS}× moiré cell")
print()

## 1. Square Lattice Overview

We create the square monolayer lattice used for the envelope basis (r = 0.43, ε = 4.02). No extremum search is performed here; k0 and band index come from prior results.

In [ ]:
# Create the base square lattice using the moire_lattice framework
square_lattice = ml.create_square_lattice(LATTICE_CONSTANT)

# Get lattice vectors
a1, a2 = square_lattice.lattice_vectors()
print(f"Square lattice vectors:")
print(f"  a1 = [{a1[0]:.4f}, {a1[1]:.4f}]")
print(f"  a2 = [{a2[0]:.4f}, {a2[1]:.4f}]")

# Generate points for visualization
lattice_points = square_lattice.generate_points(4.0)  # points within radius 4a
print(f"Generated {len(lattice_points)} lattice points for visualization")

# Calculate properties
area = square_lattice.unit_cell_area()
reciprocal_vectors = square_lattice.reciprocal_vectors()
b1, b2 = reciprocal_vectors

print(f"\nLattice properties:")
print(f"  Unit cell area: {area:.4f}")
print(f"  Reciprocal vectors:")
print(f"    b1 = [{b1[0]:.4f}, {b1[1]:.4f}]")
print(f"    b2 = [{b2[0]:.4f}, {b2[1]:.4f}]")

# Visualize the square lattice and its BZ
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Real space lattice
ax1.scatter([p[0] for p in lattice_points], [p[1] for p in lattice_points], 
           s=80, c='tab:blue', alpha=0.7, edgecolors='black')

# Add holes at lattice sites
for point in lattice_points:
    if abs(point[0]) <= 3 and abs(point[1]) <= 3:  # central region
        circle = Circle((point[0], point[1]), HOLE_RADIUS, 
                       fill=False, edgecolor='red', linewidth=2)
        ax1.add_patch(circle)

# Draw unit cell
unit_cell = np.array([[0, 0], [a1[0], a1[1]], 
                     [a1[0] + a2[0], a1[1] + a2[1]], [a2[0], a2[1]], [0, 0]])
ax1.plot(unit_cell[:, 0], unit_cell[:, 1], 'g-', linewidth=3, label='Unit cell')

ax1.set_xlim(-3, 3)
ax1.set_ylim(-3, 3)
ax1.set_aspect('equal')
ax1.set_title('Square Lattice (Real Space)')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Reciprocal space: Square BZ (Γ-X-M-Γ)
# Build first BZ square via b1,b2 half-cuts
bz = np.array([
    [0.5*b1[0]+0.5*b2[0], 0.5*b1[1]+0.5*b2[1]],
    [-0.5*b1[0]+0.5*b2[0], -0.5*b1[1]+0.5*b2[1]],
    [-0.5*b1[0]-0.5*b2[0], -0.5*b1[1]-0.5*b2[1]],
    [0.5*b1[0]-0.5*b2[0], 0.5*b1[1]-0.5*b2[1]],
    [0.5*b1[0]+0.5*b2[0], 0.5*b1[1]+0.5*b2[1]]
])
ax2.plot(bz[:, 0], bz[:, 1], 'b-', linewidth=2, label='First BZ')
ax2.fill(bz[:-1, 0], bz[:-1, 1], alpha=0.2, color='tab:blue')

# High-symmetry points
Gamma = np.array([0, 0])
X = 0.5*np.array([b1[0], b1[1]])
M = 0.5*np.array([b1[0]+b2[0], b1[1]+b2[1]])

ax2.scatter(*Gamma, s=100, c='red', marker='o', label='Γ')
ax2.scatter(*X, s=100, c='green', marker='s', label='X')
ax2.scatter(*M, s=100, c='orange', marker='^', label='M')

ax2.text(Gamma[0]+0.1, Gamma[1]+0.1, 'Γ', fontsize=12)
ax2.text(X[0]+0.1, X[1]+0.1, 'X', fontsize=12)
ax2.text(M[0]+0.1, M[1]+0.1, 'M', fontsize=12)

ax2.set_aspect('equal')
ax2.set_title('First Brillouin Zone (Square)')
ax2.set_xlabel(r'$k_x$')
ax2.set_ylabel(r'$k_y$')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Square lattice structure visualized - ready for moiré construction!")

In [ ]:
def get_candidate_from_csv(csv_path: str,
                           preferred_plot: str | None = None,
                           k_label_preference: str = 'M'):
    """Load ω0 (freq), k_frac, and band index from a candidates.csv row matching our r/eps.
    Falls back to None if not found. This avoids recomputing bands in this notebook.
    """
    import csv, os
    if not os.path.isfile(csv_path):
        return None
    rows = []
    with open(csv_path, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                r = float(row['hole_radius_a'])
                eps = float(row['eps_bg'])
                if abs(r - HOLE_RADIUS) < 5e-4 and abs(eps - DIELECTRIC_CONST) < 5e-2:
                    rows.append(row)
            except Exception:
                continue
    if not rows:
        return None
    # prefer the target band index row with label k_label_preference
    best = None
    for row in rows:
        try:
            bidx = int(row['band_index'])
            klabel = row.get('k_label', '')
            if bidx == TARGET_BAND_INDEX and (klabel == k_label_preference):
                best = row; break
        except Exception:
            pass
    if best is None:
        best = rows[0]
    # Extract
    freq = float(best['frequency'])
    kx_frac = best.get('kx_frac', '')
    ky_frac = best.get('ky_frac', '')
    if kx_frac and ky_frac:
        k_frac = (float(kx_frac), float(ky_frac))
    else:
        # Map by label on square: Γ(0,0), X(0.5,0), M(0.5,0.5)
        label = best.get('k_label', 'M')
        if label == 'Γ': k_frac = (0.0, 0.0)
        elif label == 'X': k_frac = (0.5, 0.0)
        else: k_frac = (0.5, 0.5)
    band_index = int(best['band_index'])
    return {"omega0": freq, "k_frac": k_frac, "band_index": band_index, "row": best}

candidate_csv = "/home/renlephy/msl/research/moire_cavity_exploration/pc_extrema_results_square/candidates.csv"
from_csv = get_candidate_from_csv(candidate_csv)

if from_csv is not None:
    TARGET_K_FRAC = tuple(from_csv['k_frac'])
    TARGET_BAND_INDEX = int(from_csv['band_index'])
    omega_0 = float(from_csv['omega0'])
    print("Using candidate from CSV:")
    print(f"  k(frac) = {TARGET_K_FRAC}, band = {TARGET_BAND_INDEX}, ω0 = {omega_0:.6f}")
else:
    omega_0 = None
    print("No CSV candidate found; will compute ω0 if Meep is available.")

# Minimal helper to compute ω0 and effective mass if needed

def compute_omega0_and_mass_square(a1, a2, r, eps, k_frac, band_index, resolution=RESOLUTION, num_bands=NUM_BANDS):
    if not MEEP_AVAILABLE:
        raise RuntimeError("Meep not available, cannot compute ω0 or m*.")
    # Build MPB problem for square lattice, 2D TM modes
    geom = [mp.Cylinder(radius=r * LATTICE_CONSTANT, material=mp.air, center=mp.Vector3(0, 0, 0))]
    lat = mp.Lattice(size=mp.Vector3(1, 1, 0),
                     basis1=mp.Vector3(a1[0], a1[1], 0),
                     basis2=mp.Vector3(a2[0], a2[1], 0))
    k = mp.Vector3(k_frac[0], k_frac[1], 0)
    ms = mpb.ModeSolver(geometry_lattice=lat, geometry=geom, default_material=mp.Medium(epsilon=eps),
                        k_points=[k], resolution=resolution, num_bands=num_bands, dimensions=2)
    ms.run_tm(); freqs = np.array(ms.all_freqs)[0]
    w0 = float(freqs[band_index])
    # Estimate m* by finite difference around k
    dk = 0.01
    ks = [k,
          mp.Vector3(k.x+dk, k.y, 0), mp.Vector3(k.x-dk, k.y, 0),
          mp.Vector3(k.x, k.y+dk, 0), mp.Vector3(k.x, k.y-dk, 0)]
    ms.k_points = ks; ms.run_tm(); fr = np.array(ms.all_freqs)
    w0c = fr[0, band_index]; wxp = fr[1, band_index]; wxm = fr[2, band_index]; wyp = fr[3, band_index]; wym = fr[4, band_index]
    d2x = (wxp + wxm - 2*w0c) / (dk**2)
    d2y = (wyp + wym - 2*w0c) / (dk**2)
    curv = 0.5*(d2x + d2y)
    m_eff = 1.0/curv if abs(curv) > 1e-10 else 1.0
    return w0, abs(m_eff)

if omega_0 is None and MEEP_AVAILABLE:
    omega_0, m_effective = compute_omega0_and_mass_square(a1, a2, HOLE_RADIUS, DIELECTRIC_CONST,
                                                          TARGET_K_FRAC, TARGET_BAND_INDEX)
    print(f"Computed ω0 = {omega_0:.6f} and m* = {m_effective:.4f}")
else:
    # If CSV provided only ω0, still need an estimate of m*. Compute if possible, else default 1.
    if MEEP_AVAILABLE:
        _, m_effective = compute_omega0_and_mass_square(a1, a2, HOLE_RADIUS, DIELECTRIC_CONST,
                                                        TARGET_K_FRAC, TARGET_BAND_INDEX)
        print(f"Estimated effective mass m* = {m_effective:.4f}")
    else:
        m_effective = 1.0
        if omega_0 is None: omega_0 = 0.5  # placeholder
        print("Meep unavailable; using defaults ω0≈0.5, m*=1.0 (for qualitative runs)")

## 2. Moiré Lattice Construction and Registry Analysis

Now we create the twisted bilayer moiré pattern and extract the stacking registers (AA, AB, BA).

In [ ]:
# Create the moiré pattern using the moire builder (square lattice)
print(f"🔄 Creating moiré pattern with twist angle θ = {TEST_TWIST_ANGLE_DEG:.2f}° (test run)")

TWIST_ANGLE = np.deg2rad(TEST_TWIST_ANGLE_DEG)

builder = ml.PyMoireBuilder()
builder.with_base_lattice(square_lattice)
builder.with_twist_and_scale(TWIST_ANGLE, 1.0)  # Pure rotation

moire = builder.build()

# Moiré properties
print(f"\nMoiré lattice properties:")
print(f"  Twist angle: {moire.twist_angle_degrees():.3f}°")
print(f"  Moiré period ratio: {moire.moire_period_ratio():.2f}")
print(f"  Is commensurate: {moire.is_commensurate()}")
print(f"  Moiré unit cell area: {moire.cell_area():.2f} (in units of original cell)")

# Lattices and vectors
lattice_1 = moire.lattice_1()
lattice_2 = moire.lattice_2()
moire_lattice = moire.as_lattice2d()
m1, m2 = moire_lattice.lattice_vectors()
print(f"\nMoiré lattice vectors:")
print(f"  M1 = [{m1[0]:.4f}, {m1[1]:.4f}]")
print(f"  M2 = [{m2[0]:.4f}, {m2[1]:.4f}]")
print(f"  Moiré lattice constant |M1|: {np.linalg.norm(m1):.4f}")

# Show small moiré unit cell (1×, 2×, or 3×) per user option
cells_to_show = ENVELOPE_DOMAIN_CELLS

# Generate registry detection robustly via fractional coordinates in moiré basis
# We project any point r to fractional coords (u,v) such that r = u*M1 + v*M2
# Then AA at (0 mod 1, 0 mod 1), AB at (1/2, 0) and (0, 1/2) for square (two distinct) – we label ABx, ABy
# BA coincides with AB on square; here we keep labels ABx/ABy for visual robustness.

M = np.array([[m1[0], m2[0]], [m1[1], m2[1]]])
Minv = np.linalg.inv(M)

def frac_coords(r):
    uv = Minv @ np.array([r[0], r[1]])
    return uv

# Define registry fractional targets and names for square bilayer
REGISTRY_TARGETS = [
    (np.array([0.0, 0.0]), 'AA'),
    (np.array([0.5, 0.0]), 'ABx'),
    (np.array([0.0, 0.5]), 'ABy'),
]

REG_TOL = 0.08  # tolerance in fractional space for assignment

# Build grid of candidate sites within +/- cells_to_show moiré cells
stacking_sites = []
for i in range(-cells_to_show, cells_to_show + 1):
    for j in range(-cells_to_show, cells_to_show + 1):
        cell_origin = np.array([i * m1[0] + j * m2[0], i * m1[1] + j * m2[1]])
        # canonical AA, ABx, ABy positions in this cell
        for target, name in REGISTRY_TARGETS:
            pos = cell_origin + target[0] * np.array(m1) + target[1] * np.array(m2)
            stacking_sites.append((pos, name))

# Visualization: moiré pattern and stacking map
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Plot multiple moiré unit cells outline
for i in range(-cells_to_show, cells_to_show + 1):
    for j in range(-cells_to_show, cells_to_show + 1):
        cell_origin = np.array([i * m1[0] + j * m2[0], i * m1[1] + j * m2[1]])
        cell_corners = np.array([
            cell_origin,
            cell_origin + np.array([m1[0], m1[1]]),
            cell_origin + np.array([m1[0] + m2[0], m1[1] + m2[1]]),
            cell_origin + np.array([m2[0], m2[1]]),
            cell_origin
        ])
        ax1.plot(cell_corners[:, 0], cell_corners[:, 1], 'g-', linewidth=1.0, alpha=0.7)
        ax2.plot(cell_corners[:, 0], cell_corners[:, 1], 'k-', linewidth=1.2, alpha=0.7)

# Plot lattice points in limited range
moire_size = np.linalg.norm(m1) * (cells_to_show + 0.5)
points_1 = lattice_1.generate_points(moire_size)
points_2 = lattice_2.generate_points(moire_size)

pts1 = [p for p in points_1 if abs(p[0]) <= moire_size and abs(p[1]) <= moire_size]
pts2 = [p for p in points_2 if abs(p[0]) <= moire_size and abs(p[1]) <= moire_size]

ax1.scatter([p[0] for p in pts1], [p[1] for p in pts1], s=6, c='tab:blue', alpha=0.7, edgecolors='darkblue', linewidth=0.4, label='Layer 1')
ax1.scatter([p[0] for p in pts2], [p[1] for p in pts2], s=4, c='tab:red', alpha=0.7, edgecolors='darkred', linewidth=0.4, marker='^', label='Layer 2')
ax1.set_title(f'Moiré Pattern (θ = {TEST_TWIST_ANGLE_DEG:.2f}°, {cells_to_show}× cell)')
ax1.set_aspect('equal'); ax1.legend(); ax1.grid(True, alpha=0.3)

# Stacking registry map scatter
AA = []; ABx = []; ABy = []
for pos, name in stacking_sites:
    uv = frac_coords(pos)
    # Wrap to [0,1)
    uv_wrapped = np.mod(uv, 1.0)
    # Assign label by nearest target in fractional space
    dists = [np.linalg.norm(uv_wrapped - np.mod(t, 1.0)) for t, _ in REGISTRY_TARGETS]
    idx = int(np.argmin(dists))
    label = REGISTRY_TARGETS[idx][1] if dists[idx] < REG_TOL else name
    if label == 'AA': AA.append(pos)
    elif label == 'ABx': ABx.append(pos)
    else: ABy.append(pos)

if AA:
    ax2.scatter([p[0] for p in AA], [p[1] for p in AA], s=140, c='red', alpha=0.85, edgecolors='white', linewidth=0.7, label='AA')
if ABx:
    ax2.scatter([p[0] for p in ABx], [p[1] for p in ABx], s=120, c='blue', alpha=0.85, edgecolors='white', linewidth=0.7, marker='s', label='ABx (½,0)')
if ABy:
    ax2.scatter([p[0] for p in ABy], [p[1] for p in ABy], s=120, c='green', alpha=0.85, edgecolors='white', linewidth=0.7, marker='^', label='ABy (0,½)')

ax2.set_title('Stacking Registry Map (robust)')
ax2.set_aspect('equal'); ax2.legend(); ax2.grid(True, alpha=0.3)

for ax in (ax1, ax2):
    ax.set_xlabel('x'); ax.set_ylabel('y')

plt.tight_layout(); plt.show()

print("\n🔍 Robust stacking registry sites:")
print(f"   AA:  {len(AA)}  | ABx: {len(ABx)} | ABy: {len(ABy)}")

## 3. Registry Unit Cell Construction (Square)

We now construct unit cells for each stacking registry in a square bilayer: AA, ABx (offset ½ along a1), and ABy (offset ½ along a2). In a pure twist with no dilation, ABx and ABy are symmetry-related. We will compute their band edges at the provided (n, k0) and use the differences ΔV_registry = ω_registry(k0) − ω0 to build the envelope potential.

In [ ]:
def create_registry_geometry_square(registry_type: str, hole_radius: float, a1: Tuple[float,float], a2: Tuple[float,float], layer_sep=0.0):
    # Square: offsets: AA (0,0), ABx (½,0), ABy (0,½)
    if registry_type == 'AA':
        offs = (0.0, 0.0)
    elif registry_type == 'ABx':
        offs = (0.5, 0.0)
    elif registry_type == 'ABy':
        offs = (0.0, 0.5)
    else:
        raise ValueError("registry_type must be AA|ABx|ABy for square")
    off_vec = offs[0]*np.array(a1) + offs[1]*np.array(a2)
    return [
        mp.Cylinder(hole_radius, material=mp.air, center=mp.Vector3(0, 0, 0)),
        mp.Cylinder(hole_radius, material=mp.air, center=mp.Vector3(float(off_vec[0]), float(off_vec[1]), layer_sep))
    ]


def calc_registry_edge_freq(registry_type: str, a1, a2, r, eps, k_frac, num_bands=NUM_BANDS, resolution=RESOLUTION):
    if not MEEP_AVAILABLE: return None
    geom = create_registry_geometry_square(registry_type, r*LATTICE_CONSTANT, a1, a2)
    lat = mp.Lattice(size=mp.Vector3(1, 1, 0), basis1=mp.Vector3(a1[0], a1[1], 0), basis2=mp.Vector3(a2[0], a2[1], 0))
    k = mp.Vector3(k_frac[0], k_frac[1], 0)
    ms = mpb.ModeSolver(geometry_lattice=lat, geometry=geom, default_material=mp.Medium(epsilon=eps),
                        k_points=[k], resolution=resolution, num_bands=num_bands, dimensions=2)
    ms.run_tm(); freqs = np.array(ms.all_freqs)[0]
    return freqs


registry_results = {}
if MEEP_AVAILABLE and omega_0 is not None:
    print("🎯 Calculating square registry (AA/ABx/ABy) band edges at target k...")
    for reg in ['AA','ABx','ABy']:
        fr = calc_registry_edge_freq(reg, a1, a2, HOLE_RADIUS, DIELECTRIC_CONST, TARGET_K_FRAC)
        if fr is not None:
            registry_results[reg] = fr
            print(f"  {reg}: ω[band {TARGET_BAND_INDEX}] = {fr[TARGET_BAND_INDEX]:.6f}")
    # Build ΔV per registry
    if registry_results:
        delta_omega = {reg: registry_results[reg][TARGET_BAND_INDEX] - omega_0 for reg in registry_results}
        delta_omega_AA = float(delta_omega.get('AA', 0.0))
        delta_omega_ABx = float(delta_omega.get('ABx', 0.0))
        delta_omega_ABy = float(delta_omega.get('ABy', 0.0))
else:
    # Fallback example potentials
    registry_results = {}
    delta_omega_AA = 0.0
    delta_omega_ABx = 0.015
    delta_omega_ABy = 0.015
    print("⚠️ Skipping registry band edges (no Meep); using example ΔV values.")

print("ΔV (registry) relative to ω0:")
print(f"  AA : {delta_omega_AA:+.6f}")
print(f"  ABx: {delta_omega_ABx:+.6f}")
print(f"  ABy: {delta_omega_ABy:+.6f}")

In [ ]:
# Visualize registry band comparison (square AA/ABx/ABy) for the target band
registries = ['AA', 'ABx', 'ABy']

if MEEP_AVAILABLE and len(registry_results) > 0:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    band_indices = list(range(min(6, NUM_BANDS)))
    x_pos = np.arange(len(band_indices)); width = 0.25

    for i, reg in enumerate([r for r in registries if r in registry_results]):
        freqs = registry_results[reg]
        vals = [freqs[j] for j in band_indices]
        color = 'red' if reg == 'AA' else ('blue' if reg == 'ABx' else 'green')
        ax1.bar(x_pos + i*width, vals, width, label=f'{reg} registry', alpha=0.7, color=color)

    ax1.set_xlabel('Band Index'); ax1.set_ylabel('Frequency (c/a)')
    ax1.set_title('Registry Band Frequencies at Target k₀')
    ax1.set_xticks(x_pos + width); ax1.set_xticklabels([f'Band {i+1}' for i in band_indices])
    ax1.legend(); ax1.grid(True, alpha=0.3)

    # Plot Δω for target band
    target_band_idx = int(TARGET_BAND_INDEX)
    deltas = []
    for reg in registries:
        if reg in registry_results:
            deltas.append(registry_results[reg][target_band_idx] - omega_0)
        else:
            deltas.append(np.nan)

    bars = ax2.bar(registries, deltas, color=['red','blue','green'], alpha=0.75, edgecolor='black')
    for b, d in zip(bars, deltas):
        if not np.isnan(d):
            ax2.text(b.get_x() + b.get_width()/2., d + 0.0001*np.sign(d if d!=0 else 1), f'{d:.6f}', ha='center', va='bottom' if d>=0 else 'top')

    ax2.axhline(0, color='k', lw=1, alpha=0.5)
    ax2.set_ylabel('Δω = ω_registry − ω₀ (target band)')
    ax2.set_title(f'Registry Frequency Shifts (Band {target_band_idx+1})')
    ax2.grid(True, alpha=0.3)
    plt.tight_layout(); plt.show()

    print("📈 Registry comparison plotted.")
    print(f"   ω₀ = {omega_0:.6f}, band = {target_band_idx}")
else:
    # Placeholder plot
    fig, ax = plt.subplots(figsize=(10, 6))
    deltas = [delta_omega_AA, delta_omega_ABx, delta_omega_ABy]
    colors = ['red','blue','green']
    bars = ax.bar(registries, deltas, color=colors, alpha=0.75, edgecolor='black')
    for b, d in zip(bars, deltas):
        ax.text(b.get_x() + b.get_width()/2., d + 0.0005, f'{d:.3f}', ha='center', va='bottom')
    ax.axhline(0, color='k', lw=1, alpha=0.5)
    ax.set_ylabel('Δω (placeholder)')
    ax.set_title('Registry Frequency Shifts (Placeholder)')
    ax.grid(True, alpha=0.3)
    plt.tight_layout(); plt.show()
    print("📝 Placeholder registry plot shown (Meep not available)")

## 4. Effective Hamiltonian Construction

Now we construct the effective 2D Schrödinger-like Hamiltonian using the envelope approximation. This gives us:

$$\left[-\frac{\hbar^2}{2m^*}\nabla^2 + V(\mathbf{r})\right]F(\mathbf{r}) = (\omega - \omega_0)F(\mathbf{r})$$

where:
- $m^*$ is the effective mass from the monolayer curvature
- $V(\mathbf{r})$ is the registry-dependent potential landscape  
- $F(\mathbf{r})$ is the envelope function

In [ ]:
# Effective mass and ω0 handling (integrated with provided candidate)
# If ω0 and m_effective were not set from CSV/compute, compute them now.

if 'omega_0' not in globals() or omega_0 is None or 'm_effective' not in globals():
    if MEEP_AVAILABLE:
        omega_0, m_effective = compute_omega0_and_mass_square(
            a1, a2, HOLE_RADIUS, DIELECTRIC_CONST, TARGET_K_FRAC, TARGET_BAND_INDEX,
            resolution=RESOLUTION, num_bands=NUM_BANDS
        )
        print(f"Computed ω0 = {omega_0:.6f}, m* = {m_effective:.4f}")
    else:
        omega_0 = 0.5
        m_effective = 1.0
        print("Meep unavailable; using defaults ω0≈0.5, m*=1.0")

print(f"\n🎯 Effective mass for envelope approximation: m* = {m_effective:.6f}")

In [ ]:
def create_moire_potential_square(moire, stacking_sites, delta_AA, delta_ABx, delta_ABy, grid_size=GRID_SIZE, cells=ENVELOPE_DOMAIN_CELLS):
    # Moiré vectors and domain sizing
    moire_lat = moire.as_lattice2d(); m1, m2 = moire_lat.lattice_vectors()
    L1 = np.array(m1); L2 = np.array(m2)
    # Domain linear span covers 'cells' cells centered at 0
    span_vec1 = cells * L1
    span_vec2 = cells * L2
    # Build a rectangular sampling domain aligned to m1/m2 using parameterization (u,v in [-0.5,0.5])
    # For visualization and solver, we sample on a Cartesian grid in x,y that covers this parallelogram bounding box
    # Build grid bounds
    corners = [ 0.5*span_vec1 + 0.5*span_vec2,
               -0.5*span_vec1 + 0.5*span_vec2,
               -0.5*span_vec1 - 0.5*span_vec2,
                0.5*span_vec1 - 0.5*span_vec2 ]
    corners = np.array(corners)
    xmin, ymin = corners.min(axis=0)
    xmax, ymax = corners.max(axis=0)

    x = np.linspace(xmin, xmax, grid_size)
    y = np.linspace(ymin, ymax, grid_size)
    X, Y = np.meshgrid(x, y)

    # Prepare assignment via nearest registry site (piecewise-constant potential)
    V = np.zeros_like(X)
    site_arrays = {
        'AA': np.array([pos for pos, name in stacking_sites if name == 'AA']),
        'ABx': np.array([pos for pos, name in stacking_sites if name == 'ABx']),
        'ABy': np.array([pos for pos, name in stacking_sites if name == 'ABy']),
    }
    reg_values = {'AA': delta_AA, 'ABx': delta_ABx, 'ABy': delta_ABy}

    # If any list empty, place a dummy far away to avoid edge cases
    for k in site_arrays:
        if site_arrays[k].size == 0:
            site_arrays[k] = np.array([[1e9, 1e9]])

    # Assign by nearest site type (Euclidean). Could be improved with smooth interpolation.
    for i in range(grid_size):
        for j in range(grid_size):
            p = np.array([X[i, j], Y[i, j]])
            best_label = None; best_dist = 1e99
            for label, pts in site_arrays.items():
                d = np.min(np.linalg.norm(pts - p, axis=1))
                if d < best_dist:
                    best_dist = d; best_label = label
            V[i, j] = reg_values[best_label]
    return (X, Y, V), (x, y), (L1, L2), (xmin, xmax, ymin, ymax)

print("🗺️ Creating moiré potential landscape (square)...")
(X, Y, V), (x_axis, y_axis), (L1, L2), bounds = create_moire_potential_square(
    moire, stacking_sites, delta_omega_AA, delta_omega_ABx, delta_omega_ABy,
    grid_size=GRID_SIZE, cells=ENVELOPE_DOMAIN_CELLS
)

print(f"   Grid size: {GRID_SIZE} × {GRID_SIZE}")
print(f"   Potential range: [{V.min():.6f}, {V.max():.6f}]")

# Visualizations: small moiré unit cell and potential
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Small moiré cell preview (single cell outline)
cell = np.array([[0,0], L1, L1+L2, L2, [0,0]], dtype=float)
axes[0].plot(cell[:,0], cell[:,1], 'g-', lw=2)
axes[0].set_aspect('equal'); axes[0].grid(True, alpha=0.3)
axes[0].set_title('Moiré Unit Cell (preview)'); axes[0].set_xlabel('x'); axes[0].set_ylabel('y')

# Potential map
im = axes[1].contourf(X, Y, V, levels=20, cmap='RdYlBu_r')
axes[1].contour(X, Y, V, levels=10, colors='k', alpha=0.2, linewidths=0.6)
plt.colorbar(im, ax=axes[1], label='Δω (V)')
axes[1].set_aspect('equal'); axes[1].set_title('Moiré Potential V(r)')

# 1D cross-section along a diagonal line for qualitative check
mid = GRID_SIZE//2
axes[2].plot(X[mid, :], V[mid, :], label='mid-row')
axes[2].plot(Y[:, mid], V[:, mid], label='mid-col')
axes[2].legend(); axes[2].grid(True, alpha=0.3)
axes[2].set_title('Cross-sections')

for ax in axes:
    ax.set_xlabel('x'); ax.set_ylabel('y')

plt.tight_layout(); plt.show()
print("✅ Moiré potential landscape prepared")

In [ ]:
# Build finite-difference kinetic operator with periodic BC on the rectangular sampling grid

def create_kinetic_operator(grid_size, dx, dy, m_eff):
    hbar_sq_over_2m = 1.0 / (2.0 * m_eff)
    N = grid_size * grid_size
    T = np.zeros((N, N))
    for i in range(grid_size):
        for j in range(grid_size):
            idx = i * grid_size + j
            T[idx, idx] = -hbar_sq_over_2m * (2.0/dx**2 + 2.0/dy**2)
            # neighbors with periodic wrap
            T[idx, ((i-1) % grid_size) * grid_size + j] += hbar_sq_over_2m / dx**2
            T[idx, ((i+1) % grid_size) * grid_size + j] += hbar_sq_over_2m / dx**2
            T[idx, i * grid_size + ((j-1) % grid_size)] += hbar_sq_over_2m / dy**2
            T[idx, i * grid_size + ((j+1) % grid_size)] += hbar_sq_over_2m / dy**2
    return csc_matrix(T)

print("🔧 Constructing effective Hamiltonian and solving modes...")

# Grid spacings (uniform)
dx = (x_axis[1] - x_axis[0]) if len(x_axis) > 1 else 1.0
dy = (y_axis[1] - y_axis[0]) if len(y_axis) > 1 else 1.0

T_kinetic = create_kinetic_operator(GRID_SIZE, dx, dy, m_effective)
V_flat = V.flatten(); V_potential = csc_matrix(np.diag(V_flat))
H_eff = T_kinetic + V_potential

num_modes = 10
print(f"   Hamiltonian: {H_eff.shape}, sparsity ~{100*H_eff.nnz/(H_eff.shape[0]**2):.2f}%")
print(f"   Solving for {num_modes} lowest modes...")

try:
    eigenvalues, eigenvectors = eigsh(H_eff, k=num_modes, which='SA')
    cavity_frequencies = omega_0 + eigenvalues
    print("✅ Eigenproblem solved.")
    print("Lowest few modes (Δω, ω):")
    for i in range(min(5, num_modes)):
        print(f"  {i+1}: Δω={eigenvalues[i]:.6f}, ω={cavity_frequencies[i]:.6f}")

    # Simple cavity score: negative of lowest Δω (more negative = deeper bound)
    def cavity_score(vals):
        return -float(vals[0])
    current_score = cavity_score(eigenvalues)
    print(f"Cavity score (more positive is better): {current_score:.6f}")

    # Visualize first 6 modes
    cols = 3; rows = 2
    fig, axes = plt.subplots(rows, cols, figsize=(18, 10)); axes = axes.flatten()
    for i in range(min(rows*cols, num_modes)):
        mode = eigenvectors[:, i].reshape(GRID_SIZE, GRID_SIZE)
        intensity = (mode.real**2 + mode.imag**2)
        im = axes[i].contourf(X, Y, intensity, levels=24, cmap='viridis')
        axes[i].contour(X, Y, V, levels=6, colors='white', alpha=0.6, linewidths=0.6)
        plt.colorbar(im, ax=axes[i], shrink=0.8)
        axes[i].set_title(f"Mode {i+1}: Δω={eigenvalues[i]:.4f}")
        axes[i].set_aspect('equal')
    for ax in axes:
        ax.set_xlabel('x'); ax.set_ylabel('y')
    plt.tight_layout(); plt.show()

except Exception as e:
    print(f"❌ Eigen solve failed: {e}")
    eigenvalues = None

print("Done.")

## 5. Summary and Next Steps

This refactored notebook implements the requested pipeline for the square-lattice candidate (r = 0.43, ε = 4.02):

1. Square monolayer setup with provided candidate; no extremum search here
2. Robust stacking detection (AA, ABx, ABy) and visual confirmation
3. Envelope potential from registry band-edge shifts; small moiré unit cell plotted
4. Full demonstration at test angle 1.1° with mode visualizations
5. Optional twist-angle optimization over θ to maximize cavity strength

Tips:
- Ensure Meep is installed if you want accurate ω0, m*, and ΔV registries. Otherwise, the workflow uses placeholders for qualitative checks.
- Use `ENVELOPE_DOMAIN_CELLS` to toggle between 1, 2, or 3 cell domains for the envelope solver.
- The optimization scans θ only; r and ε remain fixed to the provided candidate.

## 6. Twist-Angle Optimization

After validating the pipeline at the test angle (1.1°), we optimize the twist angle to maximize a cavity score (deeper bound → stronger cavity). We sweep using the envelope model only (fast), keeping r=0.43 and ε=4.02 fixed. The objective is the negative of the lowest eigenvalue shift (−Δω_min), i.e., larger is better.

In [ ]:
def envelope_cavity_score_for_angle(theta_deg: float,
                                     domain_cells: int = ENVELOPE_DOMAIN_CELLS,
                                     grid_size: int = max(32, GRID_SIZE//2)) -> float:
    # Build moiré for this angle
    b = ml.PyMoireBuilder(); b.with_base_lattice(square_lattice); b.with_twist_and_scale(np.deg2rad(theta_deg), 1.0)
    mo = b.build()
    mo_lat = mo.as_lattice2d(); m1_, m2_ = mo_lat.lattice_vectors()

    # Recreate stacking sites for this domain size
    stack_sites = []
    for i in range(-domain_cells, domain_cells + 1):
        for j in range(-domain_cells, domain_cells + 1):
            org = np.array([i*m1_[0] + j*m2_[0], i*m1_[1] + j*m2_[1]])
            for target, name in REGISTRY_TARGETS:
                pos = org + target[0] * np.array(m1_) + target[1] * np.array(m2_)
                stack_sites.append((pos, name))

    # Build potential (reuse ΔV from earlier)
    (Xo, Yo, Vo), (xx, yy), _, _ = create_moire_potential_square(
        mo, stack_sites, delta_omega_AA, delta_omega_ABx, delta_omega_ABy, grid_size=grid_size, cells=domain_cells
    )

    # Kinetic operator (assume dx,dy uniform)
    dxo = (xx[1]-xx[0]) if len(xx) > 1 else 1.0
    dyo = (yy[1]-yy[0]) if len(yy) > 1 else 1.0
    To = create_kinetic_operator(grid_size, dxo, dyo, m_effective)
    Vo_flat = Vo.flatten(); Vo_op = csc_matrix(np.diag(Vo_flat))
    Ho = To + Vo_op
    try:
        evals, _ = eigsh(Ho, k=4, which='SA')
        # Score: −Δω_min (bigger is better)
        return -float(evals[0])
    except Exception:
        return 0.0

if DO_TWIST_OPTIMIZATION:
    print("🔎 Running twist-angle optimization (bounds 0.3°–3.0°)...")
    res = minimize_scalar(lambda th: -envelope_cavity_score_for_angle(th),
                          bounds=(0.3, 45.0), method='bounded', options={'xatol': 0.02})
    print("Optimization result:")
    print(res)
    if res.success:
        best_angle = float(res.x)
        best_score = envelope_cavity_score_for_angle(best_angle, domain_cells=ENVELOPE_DOMAIN_CELLS, grid_size=GRID_SIZE)
        print(f"⭐ Best angle ~ {best_angle:.3f}°, score ≈ {best_score:.5f}")

        # Quick visualization at optimal angle
        print("Plotting potential and first mode at optimal angle...")
        b = ml.PyMoireBuilder(); b.with_base_lattice(square_lattice); b.with_twist_and_scale(np.deg2rad(best_angle), 1.0)
        mo_opt = b.build()
        stack_sites_opt = []
        m1o, m2o = mo_opt.as_lattice2d().lattice_vectors()
        for i in range(-ENVELOPE_DOMAIN_CELLS, ENVELOPE_DOMAIN_CELLS + 1):
            for j in range(-ENVELOPE_DOMAIN_CELLS, ENVELOPE_DOMAIN_CELLS + 1):
                org = np.array([i*m1o[0] + j*m2o[0], i*m1o[1] + j*m2o[1]])
                for t, nm in REGISTRY_TARGETS:
                    pos = org + t[0]*np.array(m1o) + t[1]*np.array(m2o)
                    stack_sites_opt.append((pos, nm))
        (Xo, Yo, Vo), (xx, yy), _, _ = create_moire_potential_square(
            mo_opt, stack_sites_opt, delta_omega_AA, delta_omega_ABx, delta_omega_ABy,
            grid_size=GRID_SIZE, cells=ENVELOPE_DOMAIN_CELLS
        )
        dxo = (xx[1]-xx[0]) if len(xx) > 1 else 1.0
        dyo = (yy[1]-yy[0]) if len(yy) > 1 else 1.0
        To = create_kinetic_operator(GRID_SIZE, dxo, dyo, m_effective)
        Ho = To + csc_matrix(np.diag(Vo.flatten()))
        ev, evec = eigsh(Ho, k=6, which='SA')
        fig, axs = plt.subplots(1, 2, figsize=(14, 6))
        im = axs[0].contourf(Xo, Yo, Vo, levels=22, cmap='RdYlBu_r'); plt.colorbar(im, ax=axs[0])
        axs[0].set_aspect('equal'); axs[0].set_title(f'V(r) at θ≈{best_angle:.3f}°')
        mode0 = evec[:, 0].reshape(GRID_SIZE, GRID_SIZE); inten0 = (mode0.real**2 + mode0.imag**2)
        im2 = axs[1].contourf(Xo, Yo, inten0, levels=24, cmap='viridis'); plt.colorbar(im2, ax=axs[1])
        axs[1].contour(Xo, Yo, Vo, levels=6, colors='white', alpha=0.6, linewidths=0.6)
        axs[1].set_aspect('equal'); axs[1].set_title(f'Lowest mode Δω={ev[0]:.4f}')
        for ax in axs:
            ax.set_xlabel('x'); ax.set_ylabel('y')
        plt.tight_layout(); plt.show()


In [ ]:
print("🎉 Moiré Cavity Exploration (Square) — Ready!")
print("=" * 60)
print()
print("📋 Recommended flow:")
print("1) Run through the cells up to the potential and eigen solve at θ=1.1° to verify visuals.")
print("2) If results look good, run the twist-angle optimization section.")
print()
print("⚙️ Configuration:")
print(f"   Lattice: square, a = {LATTICE_CONSTANT}")
print(f"   r/a = {HOLE_RADIUS}, ε = {DIELECTRIC_CONST}")
print(f"   k₀(frac) = {TARGET_K_FRAC}, band = {TARGET_BAND_INDEX}")
print(f"   Test angle = {TEST_TWIST_ANGLE_DEG}°, domain cells = {ENVELOPE_DOMAIN_CELLS}")
print(f"   Grid size = {GRID_SIZE}, MPB res = {RESOLUTION}")
print()
print("? Notes:")
print("   - Meep needed for accurate ω0, m*, and ΔV registry values.")
print("   - Placeholders used if Meep unavailable; adjust later as needed.")